In [1]:
# Import the requests library
import pandas as pd
import re

# Steam_store_data Load, transform, clean

In [2]:
#### Read in and preview Gamespot Reviews
steam_store_raw_df = pd.read_csv('original_datasets/steam.csv')
print(steam_store_raw_df.shape)
steam_store_raw_df.head()

(27075, 18)


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
# Special character in 'categories	' found in column header
steam_store_raw_df.columns=steam_store_raw_df.columns.str.replace('	','')

# Drop the 'categories', 'genres', and 'steamspy_tags' columns. 
# We have a more detailed account of the same information in the 'steamspy_tags' column
steam_store_raw_df = steam_store_raw_df.drop(['categories','genres','steamspy_tags','platforms'], axis=1)

steam_store_raw_df.head()

,appid,name,release_date,english,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,0,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,0,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,0,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,0,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,0,0,5250,288,624,415,5000000-10000000,3.99


In [4]:
# Make all names lower case
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.lower()
# Remove all special characters
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.replace(r'[^\x00-\x7f]', '')
# Remove all non-alpha-numeric
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.replace(r'[^\w ]+', '')
# Remove all duplicate whitespace
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.replace(r'\s+', ' ')
# Remove leading and trailing whitespace
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.strip(r' ')

In [5]:
# Roman numeral dict for replacement
rom_to_int_dict = {r' ii$': ' 2',
                  r' iii$': ' 3',
                  r' iii$': ' 4',
                  r' v$': ' 5',
                  r' vi$': ' 6',
                  r' vii$': ' 7',
                  r' viii$': ' 8',
                  r' ix$': ' 9',
                  r' x$': ' 10',
                  r' xi$': '11',
                  r' xii$': '12',
                  r' xiii$': '13',
                  r' xiv$': '14'}

In [6]:
# Roman numeral to integer conversion
steam_store_raw_df['name'] = steam_store_raw_df['name'].replace(rom_to_int_dict, regex=True)

In [7]:
# Create a list of values that block matching between steam and twitch for removal
remove_list = (' hd', ' remastered', ' directors cut edition', ' game of the year enhanced', 
               ' aot wings of freedom', ' aot 2', ' remastered', ' expansion', ' hd edition',
              ' board game with online multiplayer', ' gold edition', ' game of the year edition',' creative survival',
              ' ultimate edition',' complete edition',' special edition',' enhanced edition',' definitive edition',
              ' windows edition',' redux',' classic hd',' millennium edition',' deluxe edition',' steam edition',
              ' directors cut', ' the telltale series', ' a realm reborn', ' night of the raven', ' a telltale game series',
              '00db', ' bfg edition', ' sol', ' guacamelee super turbo championship edition', ' lightspeed edition',
              ' the complete edition', ' a criterion game', ' ultimate hd edition', ' hd remastered',
              ' warlord edition', ' arcade edition')

In [8]:
# Remove all unwanted values from the remove_list
steam_store_raw_df['name'] = steam_store_raw_df['name'].str.replace('|'.join(remove_list), '')

In [9]:
# Create a dictionary of values that block matching between steam and twitch for replacement
replace_dict = {'catherine classic': 'catherine', 
                'creativedestruction': 'creative destruction', 
                'eternal return: black survival': 'eternal return',
                'heroes of might and magic 3 the shadow of death': 'heroes of might and magic 3',
                'orcs must die unchained': 'orcs must die',
                'puyo puyotetris': 'puyo puyo tetris',
                'life is strange episode 1': 'life is strange',
                'resident evil 4 biohazard 4': 'resident evil 4',
                'resident evil 2 biohazard re2': 'resident evil 2',
                'resident evil 6 biohazard 6': 'resident evil 6',
                'resident evil 7 biohazard 7': 'resident evil 7 biohazard',
                'rollercoaster tycoon 2 triple thrill pack': 'rollercoaster tycoon 2',
                'romance of the three kingdoms 13 13': 'romance of the three kingdoms 13',
                'shadowverse ccg': 'shadowverse',
                'soulworker anime action mmo': 'soulworker',
                'super seducer how to talk': 'super seducer',
                'the bards tale 5 barrows deep': 'the bards tale 5',
                'puyo puyotetris': 'puyo puyo tetris',
                'x x2 remastered': 'xx2 remastered',
                'hitman 2 silent assassin': 'hitman 2',
                'hitman episode 3 marrakesh': 'hitman 3',
                'la noire the': 'la noire',
                'life is strange before the storm episode 1 awake': 'life is strange before the storm',
               'microsoft flight simulator 2004 a century of flight': 'microsoft flight simulator',
               'sid meiers civilization vi rise fall': 'sid meiers civilization vi rise and fall'}

In [10]:
# Replace all keys from the replace_dict with their values within the dataframe
steam_store_raw_df['name'] = steam_store_raw_df['name'].replace(replace_dict, regex=True)

In [11]:
# Select duplicate rows in the 'name' column
dup_rows_steam = steam_store_raw_df[steam_store_raw_df.duplicated(['name'])]
dup_rows_steam = dup_rows_steam.sort_values(by=['name'])
dup_rows_steam.head()

,appid,name,release_date,english,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
22155,855670,,2018-06-01,1,Wu Zhang,Wu Zhang,0,0,2,2,0,0,0-20000,2.09
21395,832520,,2018-07-30,0,Winturn Games,Winturn Games,0,0,15,5,0,0,0-20000,2.09
25409,970530,,2018-11-27,0,DoubleC Games,DoubleC Games,0,0,11,2,0,0,0-20000,1.69
21123,822470,,2018-09-25,0,Smm,Smm,0,0,13,3,0,0,0-20000,0.79
25475,973560,,2018-11-23,0,且御龙行,且御龙行,0,0,4,4,0,0,0-20000,15.49


In [12]:
# Drop duplicates (only duplicates above are unnamed games)
steam_store_raw_df.drop_duplicates(subset ="name", inplace = True)
print(steam_store_raw_df.shape)

(26859, 14)


In [13]:
# Add prefix for easy ID downstream
steam_store_raw_df = steam_store_raw_df.add_prefix('st_')

# twitch_raw_df Load, transform, clean

In [14]:
#### Read in and preview twitch data
twitch_raw_df = pd.read_csv('original_datasets/Twitch_game_data.csv')
print(twitch_raw_df.shape)

(12750, 12)


In [15]:
# Make all names lower case
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.lower()
# Remove all special characters
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.replace(r'[^\x00-\x7f]', '')

In [16]:
# Make all names lower case
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.lower()
# Remove all special characters
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.replace(r'[^\x00-\x7f]', '')
# Remove all non-alpha-numeric
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.replace(r'[^\w ]+', '')

# Remove all duplicate whitespace
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.replace(r'\s+', ' ')
# Remove leading and trailing whitespace
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.strip(r' ')

# Roman numeral to integer conversion
twitch_raw_df['Game'] = twitch_raw_df['Game'].replace(rom_to_int_dict, regex=True)
# Remove all unwanted values from the remove_list
twitch_raw_df['Game'] = twitch_raw_df['Game'].str.replace('|'.join(remove_list), '')
# Replace all keys from the replace_dict with their values within the dataframe
twitch_raw_df['Game'] = twitch_raw_df['Game'].replace(replace_dict, regex=True)

In [17]:
# Add prefix for easy ID downstream
twitch_raw_df = twitch_raw_df.add_prefix('tw_')

# metacritic_raw_df Load, transform, clean

In [18]:
# Read in and preview metacritic data
metacritic_raw_df = pd.read_csv('original_datasets/metacritic_game_info.csv')
print(metacritic_raw_df.shape)
metacritic_raw_df.head()

(5000, 9)


,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,1-2
2,2,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,PlayStation3,98,7.5,1 Player
3,3,SoulCalibur,1999,Namco,Action;Fighting;3D,Dreamcast,98,8.6,1-2
4,4,Grand Theft Auto IV,2008,RockstarNorth,Action Adventure;Modern;Modern;Open-World,Xbox360,98,7.9,1 Player


In [19]:
# Only keep PC game review records
metacritic_raw_df = metacritic_raw_df[metacritic_raw_df.Platform == 'PC']
print(metacritic_raw_df.shape)

(1381, 9)


In [20]:
# Make all names lower case
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.lower()
# Remove all special characters
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.replace(r'[^\x00-\x7f]', '')
# Remove all non-alpha-numeric
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.replace(r'[^\w ]+', '')

# Remove all duplicate whitespace
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.replace(r'\s+', ' ')
# Remove leading and trailing whitespace
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.strip(r' ')

# Roman numeral to integer conversion
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].replace(rom_to_int_dict, regex=True)
# Remove all unwanted values from the remove_list
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].str.replace('|'.join(remove_list), '')
# Replace all keys from the replace_dict with their values within the dataframe
metacritic_raw_df['Title'] = metacritic_raw_df['Title'].replace(replace_dict, regex=True)

In [21]:
# Remove Redundant columns
metacritic_raw_df = metacritic_raw_df.drop(['Publisher','Genre','Platform','No_Players', 'Unnamed: 0'], axis=1)
metacritic_raw_df.head()

,Title,Year,Metascore,Avg_Userscore
20,halflife 2,2004,96,9.1
21,grand theft auto 5,2015,96,7.7
28,the orange box,2007,96,9.2
34,halflife,1998,96,9.1
38,bioshock,2007,96,8.5


In [22]:
# Add prefix for easy ID downstream
metacritic_raw_df = metacritic_raw_df.add_prefix('meta_')

# steamspy_tag Load, transform, clean

In [23]:
# Read in and preview steamspy tag info
steamspy_tag_raw_df = pd.read_csv('original_datasets/steamspy_tag_data.csv')
print(steamspy_tag_raw_df.shape)
# Add prefix for easy ID downstream
steamspy_tag_raw_df = steamspy_tag_raw_df.add_prefix('st_spy_')

(29022, 372)


In [24]:
# Standardize string merge column to 'Game'
steam_store_raw_df.rename(columns = {'st_name':'Game'}, inplace=True)
metacritic_raw_df.rename(columns = {'meta_Title':'Game'}, inplace=True)
twitch_raw_df.rename(columns = {'tw_Game':'Game'}, inplace=True)

# FINAL MERGE

In [25]:
# Create list of dfs that will join on Game str
str_join_dfs = [steam_store_raw_df, metacritic_raw_df, twitch_raw_df]

In [26]:
# String merge tables join
from functools import reduce
str_merge_df = reduce(lambda left,right: pd.merge(left,right,on='Game'),str_join_dfs)

In [27]:
print(str_merge_df.shape)

(2764, 28)


In [28]:
final_df = pd.merge(left=str_merge_df, right=steamspy_tag_raw_df, left_on='st_appid', right_on='st_spy_appid')

In [29]:
print(final_df.shape)
final_df.head()

(2764, 400)


,st_appid,Game,st_release_date,st_english,st_developer,st_publisher,st_required_age,st_achievements,st_positive_ratings,st_negative_ratings,...,st_spy_warhammer_40k,st_spy_web_publishing,st_spy_werewolves,st_spy_western,st_spy_word_game,st_spy_world_war_i,st_spy_world_war_ii,st_spy_wrestling,st_spy_zombies,st_spy_e_sports
0,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,0,0,0,0,0,0,607,0
1,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,0,0,0,0,0,0,607,0
2,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,0,0,0,0,0,0,607,0
3,440,team fortress 2,2007-10-10,1,Valve,Valve,0,520,515879,34036,...,0,0,0,0,0,0,0,0,0,0
4,440,team fortress 2,2007-10-10,1,Valve,Valve,0,520,515879,34036,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#put all column names to a list for future for loop use to go thru each of the columns in the list
column_names_list = final_df.columns.to_list()

In [33]:
#create an empty list to have the columns added to it (for dropping later)
li = []

#create a function to add in columns that's meeting a certain criteria
def columns_ready_to_drop(col_name):
    #if column value type is int, continue the following, if not, ignore.
    if final_df[col_name].dtypes == "int64":

        #if dataframe col_name has 0 values more than 80% of the number of rows, save in a list, and ready to drop
        non_zero_rows = final_df[col_name].apply(lambda r: True if r > 0 else False)
        numberOfRows = len(non_zero_rows[non_zero_rows == True].index)
        #testing
        #print(numberOfRows)
        #When the percentage below is increasing, it means the column has to have less zero rows to be able to remain in the new database. 
        # 0.2 means if non-zero rows are only 20% or less of the total rows, the column will be add to the drop list.
        if numberOfRows <= 0.20 *len(final_df):
            li.append(col_name)
        else:
            pass
    else:
        pass
    return li

In [34]:
# use the function to run thru all columns
for col in column_names_list:
    columns_ready_to_drop(col)
    
print(li)

['st_spy_1980s', 'st_spy_1990s', 'st_spy_2.5d', 'st_spy_2d', 'st_spy_2d_fighter', 'st_spy_360_video', 'st_spy_3d', 'st_spy_3d_platformer', 'st_spy_3d_vision', 'st_spy_4_player_local', 'st_spy_4x', 'st_spy_6dof', 'st_spy_atv', 'st_spy_abstract', 'st_spy_action_adventure', 'st_spy_addictive', 'st_spy_agriculture', 'st_spy_aliens', 'st_spy_alternate_history', 'st_spy_america', 'st_spy_animation_&_modeling', 'st_spy_anime', 'st_spy_arcade', 'st_spy_arena_shooter', 'st_spy_artificial_intelligence', 'st_spy_assassin', 'st_spy_asynchronous_multiplayer', 'st_spy_audio_production', 'st_spy_bmx', 'st_spy_base_building', 'st_spy_baseball', 'st_spy_based_on_a_novel', 'st_spy_basketball', 'st_spy_batman', 'st_spy_battle_royale', 'st_spy_beat_em_up', 'st_spy_beautiful', 'st_spy_benchmark', 'st_spy_bikes', 'st_spy_blood', 'st_spy_board_game', 'st_spy_bowling', 'st_spy_building', 'st_spy_bullet_hell', 'st_spy_bullet_time', 'st_spy_crpg', 'st_spy_capitalism', 'st_spy_card_game', 'st_spy_cartoon', 'st_s

In [37]:
#dropping columns when the dtypes of the columns is int64 and there are more than 80% of values being 0.
# tried 80%, 50%, 45% non-zero rows, but it's dropping too many columns, so chose 20% as the cut off for now, this can be changed.
final_df = final_df.drop(columns=li)
final_df

,st_appid,Game,st_release_date,st_english,st_developer,st_publisher,st_required_age,st_achievements,st_positive_ratings,st_negative_ratings,...,st_spy_replay_value,st_spy_sandbox,st_spy_shooter,st_spy_simulation,st_spy_singleplayer,st_spy_story_rich,st_spy_strategy,st_spy_tactical,st_spy_team_based,st_spy_third_person
0,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,1049,0,1315,1116,0,0,0,0
1,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,1049,0,1315,1116,0,0,0,0
2,220,halflife 2,2004-11-16,1,Valve,Valve,0,33,67902,2419,...,0,0,1049,0,1315,1116,0,0,0,0
3,440,team fortress 2,2007-10-10,1,Valve,Valve,0,520,515879,34036,...,0,0,7857,0,0,0,0,2726,5363,0
4,440,team fortress 2,2007-10-10,1,Valve,Valve,0,520,515879,34036,...,0,0,7857,0,0,0,0,2726,5363,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,729040,borderlands,2019-04-03,1,Gearbox Software;Blind Squirrel Games,2K,18,80,1648,920,...,0,0,0,0,0,0,0,0,0,0
2760,764790,the messenger,2018-08-30,1,Sabotage,Devolver Digital,0,40,1846,90,...,0,0,0,0,24,0,0,0,0,0
2761,764790,the messenger,2018-08-30,1,Sabotage,Devolver Digital,0,40,1846,90,...,0,0,0,0,24,0,0,0,0,0
2762,845070,the awesome adventures of captain spirit,2018-06-25,1,DONTNOD Entertainment,Square Enix,0,0,8077,667,...,0,0,0,0,54,99,0,0,0,0


In [38]:
print(final_df.shape)
final_df.to_csv('for_ml_model.csv')

(2764, 57)
